In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d 'vipoooool/new-plant-diseases-dataset'

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
100% 2.69G/2.70G [00:35<00:00, 110MB/s] 
100% 2.70G/2.70G [00:35<00:00, 81.3MB/s]


In [ ]:
import zipfile
dataset_zip=zipfile.ZipFile('/content/new-plant-diseases-dataset.zip','r')
dataset_zip.extractall()
dataset_zip.close()

In [ ]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Path ke folder dataset
train_path = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_path = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'


# Fungsi untuk membaca dataset
def load_dataset(folder_path):
    features = []
    labels = []
    for class_label in os.listdir(folder_path):
        class_folder = os.path.join(folder_path, class_label)
        if os.path.isdir(class_folder):
            for img_file in os.listdir(class_folder):
                img_path = os.path.join(class_folder, img_file)
                img = cv2.imread(img_path)
                if img is not None:
                    # Resize gambar agar seragam (misalnya, 64x64)
                    img = cv2.resize(img, (64, 64))
                    features.append(img)
                    labels.append(class_label)
    return np.array(features), np.array(labels)

# Load data train dan validasi
X_train, y_train = load_dataset(train_path)
X_valid, y_valid = load_dataset(valid_path)


In [ ]:
# Encode label menggunakan LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)


In [ ]:
# Fungsi untuk mengekstrak fitur sederhana dari gambar
def extract_features(images):
    extracted_features = []
    for img in images:
        # Rata-rata dan standar deviasi untuk tiap kanal (R, G, B)
        mean = np.mean(img, axis=(0, 1))
        std = np.std(img, axis=(0, 1))
        features = np.concatenate((mean, std))
        extracted_features.append(features)
    return np.array(extracted_features)

# Ekstraksi fitur untuk data train dan validasi
X_train_features = extract_features(X_train)
X_valid_features = extract_features(X_valid)


In [ ]:
# Inisialisasi dan latih model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_features, y_train_encoded)

# Prediksi data validasi
y_pred = rf_model.predict(X_valid_features)


In [ ]:
# Evaluasi hasil
print("Classification Report:")
print(classification_report(y_valid_encoded, y_pred, target_names=label_encoder.classes_))
print("Accuracy Score:", accuracy_score(y_valid_encoded, y_pred))


Classification Report:
                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       0.86      0.87      0.86       504
                                 Apple___Black_rot       0.76      0.86      0.81       497
                          Apple___Cedar_apple_rust       0.84      0.87      0.86       440
                                   Apple___healthy       0.88      0.86      0.87       502
                               Blueberry___healthy       0.78      0.77      0.77       454
          Cherry_(including_sour)___Powdery_mildew       0.82      0.87      0.85       421
                 Cherry_(including_sour)___healthy       0.95      0.98      0.96       456
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.84      0.87      0.86       410
                       Corn_(maize)___Common_rust_       0.94      0.99      0.96       477
               Corn_(maize)___Northern_Leaf_Blight      